In [2]:
import json

def load_simple_json(filename):
    with open(filename, 'r') as f:
        return json.load(f)


l="ru"
m_no_page=load_simple_json(l+'/wiki/m_no_page.json')
m_redirection=load_simple_json(l+'/wiki/m_redirection.json')
m_page=load_simple_json(l+'/wiki/m_page.json')
wiki_pages=load_simple_json(l+'/wiki/berufe_wiki.json')

j=0
k=0
k1=0
n=0
for i in wiki_pages:#632
    if i in m_page:
        j+=1
    elif i in m_redirection:
        k+=1
    elif i in m_redirection.values():
        k1+=1
    elif i in m_no_page:
        n+=1
print j,k,k1,n #135,0 from 632   #130,0,0 from 632  #135,0,23,0 from 632



m=0
k=0
substr_m_wiki={}
substr_wiki_m={}
for i in m_no_page:
    for j in wiki_pages:
        if i in j:#if some m_no_page is a substring of berufe_wiki 
            m+=1
            print i,j
            if substr_m_wiki.has_key(i):
                substr_m_wiki[i].append(j)
            else:
                substr_m_wiki[i]=[j]
        if j in i: #if some berufe_wiki is a substring of m_no_page  
            k+=1 
            if substr_wiki_m.has_key(i):
                substr_wiki_m[i].append(j)
            else:
                substr_wiki_m[i]=[j]
            
print m,k #6,1203


135 0 23 0
Электромеханик Электромеханик СЦБ
Телефонист Телефонистка
Чистильщик Чистильщик обуви
Генеральный прокурор Генеральный прокурор Российской Федерации
Генеральный прокурор Генеральный прокурор Туркмении
Генеральный прокурор Генеральный прокурор СССР
6 1203


In [11]:
#Levenshtein
import Levenshtein

print "With Levenshtein distance<=2 or ratio>0.8"
both6={}
s=0
r=0
d=0
for i in m_no_page:
    for j in wiki_pages: 
                    if i in j:
                        print i,"=?",j
                        if i not in both6.keys():
                            both6[i]=[j]
                        else:
                            both6[i].append(j)
                        s+=1
                    elif Levenshtein.ratio(i, j)>0.8:
                        print "===>",i,"==",j
                        r+=1
                        if i not in both6.keys():
                            both6[i]=[j]
                        else:
                            both6[i].append(j)
                    elif Levenshtein.distance(i, j)<=2:
                        d+=1
                        print i,":",j    
                        if i not in both6.keys():
                            both6[i]=[j]
                        else:
                            both6[i].append(j)
print s,r,d


With Levenshtein distance<=2 or ratio>0.8
Сборщик : Сварщик
===> Техник по контрольно-измерительным приборам и автоматике == Слесарь по контрольно-измерительным приборам и автоматике
===> Строгальщик == Стропальщик
===> Инженер-протезист == Инженер-геодезист
===> Инженер-протезист == Инженер-прочнист
Содовщик : Садовник
Жаровщик : Часовщик
===> Администратор зала == Администратор
===> Администратор зала == Администратор защиты
Перемотчик : Переводчик
Бандажник : Биндюжник
===> Инженер-контролер == Инженер-конструктор
===> Заведующий фотолабораторией == Заведующий лабораторией
Электромеханик =? Электромеханик СЦБ
Учетчик : Сметчик
===> Доверенный врач == Военный врач
===> Помощник машиниста тепловоза == Помощник машиниста локомотива
===> Инженер по контрольно-измерительным приборам и автоматике == Слесарь по контрольно-измерительным приборам и автоматике
===> Администратор кино == Администратор
Телефонист =? Телефонистка
Плавильщик : Плакальщик
===> Витражист == Визажист
Раклист : Радис

In [12]:
print "======Masculine_no_pages in list of pages from category Beruf (substring inclusion):======"
for i in substr_m_wiki:
    for j in substr_m_wiki[i]:
        if j in m_page:
            print "!!!already in pages",j
        elif j in m_redirection:
            print "!!!links to smth",j
        elif j in m_redirection.values():
            print "!!!links to smth value",j
            

======Masculine_no_pages in list of pages from category Beruf (substring inclusion):======
!!!already in pages Генеральный прокурор Российской Федерации
!!!already in pages Чистильщик обуви


In [13]:

def check_pages_in_list(both6):
    new_both6={}
    for i in both6:
        if i not in new_both6.keys():
            new_both6[i]=both6[i]
        for j in both6[i]:
            if j in m_page:
                index=int(both6[i].index(j))
                new_both6[i][index]=both6[i][index],1 #1=page exist
            elif j in m_redirection:
                index=int(both6[i].index(j))
                new_both6[i][index]=both6[i][index],3 #3=redirects to smth
            elif j in m_redirection.values():
                index=int(both6[i].index(j))
                new_both6[i][index]=both6[i][index],4 #4=redirects to smth value
            else:
                index=int(both6[i].index(j))
                new_both6[i][index]=both6[i][index],0 #no page
    return new_both6
new_both6=check_pages_in_list(both6)

In [15]:
l="ru"
with open(l+'/wiki/berufe_levenshtein.json', 'w') as f: #m_no_page:[wiki_pages_in_beruf_category]
    json.dump(new_both6,  f,  indent=4)#,ensure_ascii=False)
with open(l+'/wiki/berufe_substring.json', 'w') as f: #m_no_page:[wiki_pages_in_beruf_category_withsubstring_inclusion]
    json.dump(substr_wiki_m,  f,  indent=4)#,ensure_ascii=False)

In [16]:
for i in new_both6:
    print i
    for j in new_both6[i]:
        print j[0],j[1] 
    print "---------"

Рамщик
Ямщик 0
---------
Наладчик контрольно-измерительных приборов и автоматики
Слесарь по контрольно-измерительным приборам и автоматике 1
---------
Содовщик
Садовник 1
---------
Помощник машиниста тепловоза
Помощник машиниста локомотива 0
---------
Жаровщик
Часовщик 0
---------
Врач - судебно-медицинский эксперт
Судебно-медицинский эксперт 0
---------
Таксировщик
Тестировщик 0
Татуировщик 0
---------
Инженер по контрольно-измерительным приборам и автоматике
Слесарь по контрольно-измерительным приборам и автоматике 1
---------
Чистильщик
Чистильщик обуви 1
---------
Кинофотооператор
Кинооператор 1
---------
Инженер-протезист
Инженер-геодезист 0
Инженер-прочнист 0
---------
Телеоператор-постановщик
Оператор-постановщик 0
---------
Слесарь-сборщик радиоэлектронной аппаратуры и приборов
Монтажник радиоэлектронной аппаратуры и приборов 1
---------
Ведущий программы
Ведущий программист 0
---------
Волочильщик
Холодильщик 0
---------
Плавильщик
Плакальщик 0
---------
Синильщик
Сигнальщик 0

In [20]:
#for i in page_m:
#    print i
for i in m_redirection:
    print i, "redirects to ===>",m_redirection[i]

Массажист redirects to ===> Массаж
Уполномоченный по правам человека redirects to ===> Уполномоченный по правам человека в Российской Федерации
Транспортировщик redirects to ===> Транспортёр
Риэлтер redirects to ===> Риелтор
Микробиолог redirects to ===> Микробиология
Овощевод redirects to ===> Овощеводство
Картограф redirects to ===> Картография
Цветовод redirects to ===> Цветоводство
Математик redirects to ===> Математика
Стенографистка redirects to ===> Стенография
Миколог redirects to ===> микология
Акклиматизатор redirects to ===> Птичник
Фотокорреспондент redirects to ===> Фотожурналистика
Синоптик redirects to ===> Синоптика
Промышленный альпинист redirects to ===> Промышленный альпинизм
Искусствовед redirects to ===> Искусствоведение
Логопед redirects to ===> Логопедия
Лектор redirects to ===> Лекция
Технолог redirects to ===> Инженер-технолог
Пескоструйщик redirects to ===> Абразивоструйщик
Архивариус redirects to ===> Архивист
Редактор карт redirects to ===> Редактор уровней
